# Export Outputs to JSON

In this guide, we will demonstrate how to export the check's and suite's output to JSON format.
This enables export the results as a serializeable object that can later be shared or reviewed in other Python scopes.

Notebook Structure:
 
- [Load Data and Run Check](#Load-Data)
- [Export Check to JSON](#Export-a-Check's-Output-(CheckResult)-to-a-JSON-Format)
- [Display Check from JSON](#Display-the-CheckResult-Output-from-a-JSON)
- [Export Suite to JSON](#Exporting-a-Suite's-Output-(SuiteResult)-to-JSON)

## Load Data 

In [39]:
from deepchecks.datasets.classification import iris

train_dataset, test_dataset = iris.load_data()
model = iris.load_fitted_model()

## Run a Check

In [40]:
from deepchecks.checks import ModelErrorAnalysis

result = ModelErrorAnalysis().add_condition_segments_performance_relative_difference_not_greater_than(\
                                ).run(train_dataset, test_dataset, model)

### Observe CheckResult Display and Value

In [41]:
result

Status,Condition,More Info
!,The performance difference of the detected segments must not be greater than 5%,"Found change in Accuracy in features above threshold: {'petal length (cm)': '8%', 'petal width (cm)': '8%'}"


In [42]:
# and value can be observed
result.value

{'scorer_name': 'Accuracy',
 'feature_segments': {'petal length (cm)': {'segment1': {'score': 0.92,
    'n_samples': 25,
    'frac_samples': 0.6578947368421053},
   'segment2': {'score': 1.0,
    'n_samples': 13,
    'frac_samples': 0.34210526315789475}},
  'petal width (cm)': {'segment1': {'score': 0.92,
    'n_samples': 25,
    'frac_samples': 0.6578947368421053},
   'segment2': {'score': 1.0,
    'n_samples': 13,
    'frac_samples': 0.34210526315789475}}}}

## Export a Check's Output (CheckResult) to a JSON Format 

Serialization of the output to a JSON format is possible using the `to_json` function. This function takes the check outputs and serializes it to a JSON string.

The format of the check's output json includes all of the info required to reconstruct the check run and it's output: it's name, the parameters the check receives, one sentence summary of the check's purpose, it's result value and a json of the data relevant for displaying the check's outputs 

### See JSON Structure

In [43]:
from deepchecks import CheckResult

help(CheckResult.to_json)

Help on function to_json in module deepchecks.base.check:

to_json(self, with_display: bool = True)
    Return check result as json.
    
    Args:
        with_display (bool): controls if to serialize display or not
    
    Returns:
        json in the format:
        {'name': .., 'params': .., 'header': ..,
         'summary': .., 'conditions_table': .., 'value', 'display': ..}



In [44]:
# get output JSON
serialized_output = result.to_json()

In [45]:
import json

# note - conditions_table key exists only if there are conditions that were defined on check
json.loads(serialized_output).keys()

dict_keys(['name', 'params', 'header', 'summary', 'conditions_table', 'value', 'display'])

### Observe JSON Output

In [46]:
json.loads(serialized_output)

{'name': 'Model Error Analysis',
 'params': {},
 'header': 'Model Error Analysis',
 'summary': 'Find features that best split the data into segments of high and low model error.',
 'conditions_table': '[{"Status":"<div style=\\"color: orange;text-align: center;font-weight:bold\\">!<\\/div>","Condition":"The performance difference of the detected segments must not be greater than 5%","More Info":"Found change in Accuracy in features above threshold: {\'petal length (cm)\': \'8%\', \'petal width (cm)\': \'8%\'}"}]',
 'value': {'scorer_name': 'Accuracy',
  'feature_segments': {'petal length (cm)': {'segment1': {'score': {'py/reduce': [{'py/function': 'numpy.core.multiarray.scalar'},
       {'py/tuple': [{'py/reduce': [{'py/type': 'numpy.dtype'},
           {'py/tuple': ['f8', False, True]},
           {'py/tuple': [3, '<', None, None, None, -1, -1, 0]}]},
         {'py/b64': 'cT0K16Nw7T8='}]}]},
     'n_samples': 25,
     'frac_samples': 0.6578947368421053},
    'segment2': {'score': {'py

### Display the CheckResult Output from a JSON

The serialized JSON can be used to reproduce the run in other Python scopes, by using the `display_from_json` function

In [47]:
CheckResult.display_from_json(serialized_output)

Model Error Analysis

Find features that best split the data into segments of high and low model error.

Conditions Summary

Status,Condition,More Info
!,The performance difference of the detected segments must not be greater than 5%,"Found change in Accuracy in features above threshold: {'petal length (cm)': '8%', 'petal width (cm)': '8%'}"


Additional Outputs

The following graphs show the distribution of error for top features that are most useful for distinguishing
 high error samples from low error samples. Top features are calculated using `feature_importances_`.

## Exporting a Suite's Output (SuiteResult) to JSON

### Running a Suite

In [48]:
from deepchecks.suites import full_suite

suite = full_suite()

In [49]:
suite_result = suite.run(train_dataset=train_dataset, test_dataset=test_dataset, model=model)
suite_json = suite_result.to_json()

Full Suite:   0%|          | 0/36 [00:00<?, ? Check/s]

### Observe Suite JSON Strucutre

In [50]:
help(SuiteResult.to_json)

Help on function to_json in module deepchecks.base.suite:

to_json(self, with_display: bool = True)
    Return check result as json.
    
    Args:
        with_display (bool): controls if to serialize display of checks or not
    
    Returns:
        {'name': .., 'results': ..}



Suite name:

In [51]:
json.loads(serialized_suite)['name']

'Full Suite'

Results is an array of CheckResult jsons, let's see how many checks ran in suite:

In [52]:
len(json.loads(serialized_suite)['results'])

52

Lets observe a specific check's output, which is similar to the JSON we saw before for each check 

In [53]:
json.loads(json.loads(serialized_suite)['results'][0])

{'name': 'Model Info',
 'params': {},
 'header': 'Model Info',
 'summary': 'Summarize given model parameters.',
 'value': {'type': 'RandomForestClassifier',
  'params': {'bootstrap': True,
   'ccp_alpha': 0.0,
   'class_weight': None,
   'criterion': 'gini',
   'max_depth': None,
   'max_features': 'auto',
   'max_leaf_nodes': None,
   'max_samples': None,
   'min_impurity_decrease': 0.0,
   'min_samples_leaf': 1,
   'min_samples_split': 2,
   'min_weight_fraction_leaf': 0.0,
   'n_estimators': 100,
   'n_jobs': None,
   'oob_score': False,
   'random_state': None,
   'verbose': 0,
   'warm_start': False}},
 'display': [{'py/tuple': ['html', 'Model Type: RandomForestClassifier']},
  {'py/tuple': ['dataframe',
    '[{"Parameter":"bootstrap","Value":true,"Default":true},{"Parameter":"ccp_alpha","Value":0.0,"Default":0.0},{"Parameter":"class_weight","Value":null,"Default":null},{"Parameter":"criterion","Value":"gini","Default":"gini"},{"Parameter":"max_depth","Value":null,"Default":null},